In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import openai
import json
import yaml
from graph_utils import load_graph

In [36]:
def load_problem_tags(problem_yaml, problem):
    with open(problem_yaml, "r") as fp:
        problem_tags = yaml.safe_load(fp)[problem]["tags"]
    problem_tags = ["- "+str(item).replace("{","").replace("}","").replace("'","") 
                    for item in problem_tags]
    return problem_tags

def load_edge_prompts(graph):
    edge_prompts = []
    for edge in graph.edges:
        edge_prompts.append({
            "prompt_A": edge.prompt_from.strip(),
            "prompt_B": edge.prompt_to.strip(),
            "diff": edge.diff.strip(),
            "edits":  "\n".join(["- "+str(item).replace("{","").replace("}","").replace("'","") 
                    for item in edge._edge_tags])
        })
    return edge_prompts

In [37]:
graph_yaml = "../tagging_clues/tagging_graphs_sep27/add_int.yaml"
problem_tags_yaml = "../tagging_clues/problem_clues.yaml"

graph = load_graph(graph_yaml)
problem_tags = load_problem_tags(problem_tags_yaml, graph.problem)
edge_prompts = load_edge_prompts(graph)
print(problem_tags)
print(json.dumps(edge_prompts[0], indent=4))

['- 1: edge case of list in list', '- 2: concatenate num to strings', '- 3: add num to integers', '- 4: return list']
{
    "prompt_A": "def add_int(lst, num):\n    \"\"\"\n    Takes an input list (which can have both strings and integers) and an integer. Adds the input integer to each item in the input list. \n    \"\"\"",
    "prompt_B": "def add_int(lst, num):\n    \"\"\"\n    Takes an input list and an input integer. For all strings in the list, the input integer is concatenated after the end of the string. For all numbers in the list, the input integer is added to that number.\n    \"\"\"",
    "diff": "--- \n+++ \n@@ -6,25 +6,39 @@\n an input list-(which-can-have-both-strings-and-integers) and an+input integer.-Adds+For+all+strings+in+the+list, the input integer-to-each-item+is+concatenated+after+the+end+of+the+string.+For+all+numbers in the+list,+the input-list.+integer+is+added+to+that+number. \"\"\"",
    "edits": "- a2\n- m3"
}


In [51]:
prompt = """
You are given two function docstrings, A and B, written by a student. The
student starts with A and modifies it to get B. Select out of the following 
edit types, the changes that the student has applied:

- 0: no changes or trivial change with no semantic difference
{problem_tags}

Further, add a prefix specifying HOW the student has applied the change.
Use the following modifiers.

- a: student has added the edit
- d: student has removed the edit
- l: student has revised the edit with less detail
- m: student has revised the edit with more detail
""".strip()

example = """
# prompt A
{prompt_A}

# prompt B
{prompt_B}

# edits applied
{edits}
""".strip()

query = example.split("# edits applied\n")[0] + "# edits applied\n"

In [58]:
prompt_start = prompt.format(problem_tags="\n".join(problem_tags))
prompt_example_0 = example.format(**edge_prompts[0])
prompt_cont = query.format(**edge_prompts[7])

full_prompt = "\n\n".join([p.strip() for p in [prompt_start, prompt_cont]])
print(full_prompt)
print("**EXPECTED**")
print(edge_prompts[7]["edits"])

You are given two function docstrings, A and B, written by a student. The
student starts with A and modifies it to get B. Select out of the following 
edit types, the changes that the student has applied:

- 0: no changes or trivial change with no semantic difference
- 1: edge case of list in list
- 2: concatenate num to strings
- 3: add num to integers
- 4: return list

Further, add a prefix specifying HOW the student has applied the change.
Use the following modifiers.

- a: student has added the edit
- d: student has removed the edit
- l: student has revised the edit with less detail
- m: student has revised the edit with more detail

# prompt A
def add_int(lst, num):
    """
    Return a new list with the integer added at each index of the old list. If it is a string, append it to the end. If it is an integer, concatenate.
    """

# prompt B
def add_int(lst, num):
    """
    Return a new list with the integer added at each index of the old list. If it is a string, add it to the e

In [67]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Answer concisely."},
        {
            "role": "user",
            "content": full_prompt
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='- m3: The student has revised the description with more detail to explain that strings will be combined to create a new string.\n- a2: The student has added that if the input is a string containing an integer, concatenate.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [68]:
print(completion.choices[0].message.content)

- m3: The student has revised the description with more detail to explain that strings will be combined to create a new string.
- a2: The student has added that if the input is a string containing an integer, concatenate.
